In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import HTML, display
import os
from time import perf_counter
from IPython.display import display, HTML
import random
from sqlalchemy import create_engine
from dotenv import load_dotenv

# tqdm 설정 (진행률 확인용)
from tqdm import tqdm
tqdm.pandas()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
load_dotenv()

# 1. DB 연결 정보 설정 (아까 알려주신 정보 기준)
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME') # 혹은 "musinsa_db" (둘 중 실제 스키마 이름으로 작성)

# 2. 엔진 생성 (pymysql 드라이버 사용)
db_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

print("엔진 생성 완료.")

In [ ]:
master = np.load(r"backend\master_data.npz")

In [ ]:
(master)

numpy.lib.npyio.NpzFile

In [11]:
# 데이터 로드
master = np.load(r"backend\master_data.npz",allow_pickle=True)

def check_missing(name, vecs):
    # 각 행(상품)별로 모든 벡터 원소의 절대값 합을 구함
    # 합이 0이면 해당 상품은 벡터 데이터가 없는 (0으로 채워진) 상태임
    row_sums = np.sum(np.abs(vecs), axis=1)
    missing_count = np.sum(row_sums == 0)
    total_count = len(row_sums)
    missing_rate = (missing_count / total_count) * 100
    
    print(f"--- [{name}] 검사 결과 ---")
    print(f"✅ 정상 데이터: {total_count - missing_count}개")
    print(f"❌ 누락 데이터: {missing_count}개")
    print(f"📊 누락율: {missing_rate:.2f}%")
    print("")

# 각 벡터 항목별로 누락 확인
check_missing("상품명 벡터 (name_vecs)", master['name_vecs'])
check_missing("브랜드 벡터 (brand_vecs)", master['brand_vecs'])
check_missing("이미지 벡터 (img_vecs)", master['img_vecs'])
check_missing("카테고리 벡터 (cat_vecs)", master['cat_vecs'])

--- [상품명 벡터 (name_vecs)] 검사 결과 ---
✅ 정상 데이터: 211311개
❌ 누락 데이터: 0개
📊 누락율: 0.00%

--- [브랜드 벡터 (brand_vecs)] 검사 결과 ---
✅ 정상 데이터: 211311개
❌ 누락 데이터: 0개
📊 누락율: 0.00%

--- [이미지 벡터 (img_vecs)] 검사 결과 ---
✅ 정상 데이터: 210703개
❌ 누락 데이터: 608개
📊 누락율: 0.29%

--- [카테고리 벡터 (cat_vecs)] 검사 결과 ---
✅ 정상 데이터: 211311개
❌ 누락 데이터: 0개
📊 누락율: 0.00%



In [ ]:
# 필요한 컬럼만 가져오기 (브랜드 정보 JOIN 추가)
# Collation 충돌 방지를 위해 COLLATE utf8mb4_unicode_ci 추가
query2 = """
    SELECT
        p.product_id,
        p.product_name,
        p.img_url,
        b.korea_name
    FROM products as p
    LEFT JOIN brands b
        ON p.brand COLLATE utf8mb4_unicode_ci = b.korea_name COLLATE utf8mb4_unicode_ci
"""
df = pd.read_sql(query2, engine)

In [ ]:
df

In [ ]:
def attach_embedding(df, path, col_name):
    data = np.load(path)
    emb_df = pd.DataFrame({
        "product_id": data["ids"],
        col_name: list(data["embeddings"])
    })
    return df.merge(emb_df, on="product_id", how="left")


In [ ]:
df = attach_embedding(
    df,
    r"backend\image_embedding\top_embedded.npz",
    "img_emb"
)

df = attach_embedding(
    df,
    r"sentence_embedding\product_embeddings.npz",
    "product_emb"
)

df = attach_embedding(
    df,
    r"sentence_embedding\category_embeddings.npz",
    "category_emb"
)

df = attach_embedding(
    df,
    r"sentence_embedding\brand_desc_embeddings.npz",
    "brand_desc_emb"
)

df = attach_embedding(
    df,
    r"sentence_embedding\brand_cat_embeddings.npz",
    "brand_cat_emb"
)


In [ ]:
df

In [ ]:
from time import perf_counter
from IPython.display import display, HTML

# =====================================================
# 1. 설정 및 이미지 임베딩 로드
# =====================================================
TARGET_ID = 4302685 # 기준 상품 ID
# TARGET_ID = 3575420
# TARGET_ID = 5360472

start_total = perf_counter()

# 이미지 임베딩 로드 및 정규화
img_npz = emb_top_img
img_ids = img_npz["ids"]
img_embs = img_npz["embeddings"]
img_embs = img_embs / (np.linalg.norm(img_embs, axis=1, keepdims=True) + 1e-9)
img_id_to_idx = {pid: i for i, pid in enumerate(img_ids)}

# TARGET_ID 유효성 검사 (없으면 첫 번째 ID로 대체)
if TARGET_ID not in img_id_to_idx:
    print(f"TARGET_ID({TARGET_ID})가 임베딩에 없어 '{img_ids[0]}'로 변경합니다.")
    TARGET_ID = img_ids[0]

# =====================================================
# 2. 대표 상품 인덱스 및 기준 벡터 확보
# =====================================================
# df 내의 product_id와 비교하여 인덱스 추출
target_idx_arr = df.index[df["product_id"].astype(str).str.strip() == str(TARGET_ID).strip()]
if len(target_idx_arr) == 0:
    raise ValueError(f"ID {TARGET_ID}가 메타데이터(df)에 존재하지 않습니다.")
target_idx = target_idx_arr[0]

# 이미지 기준 벡터
target_img_vec = img_embs[img_id_to_idx[TARGET_ID]]
cos_image_all = img_embs @ target_img_vec

for emb_col in [
    "img_emb",
    "product_emb",
    "brand_emb",
    "style_emb",
    "category_emb",
    "brand_desc_emb",
    "brand_cat_emb",
]:
    if emb_col in df.columns:
        lens = df[emb_col].apply(
            lambda x: len(x) if isinstance(x, (list, np.ndarray)) else -1
        )
        print(f"\n[{emb_col}] 길이 분포")
        print(lens.value_counts().head(5))
# =====================================================
# 3. 텍스트 임베딩 컬럼별 코사인 유사도 계산
# =====================================================
EMB_DIMS = {
    "img_emb":512,
    "product_emb": 200,
    "brand_cat_emb": 200,
    "category_emb": 50,
    "brand_desc_emb": 768,
    "brand_emb": 50,
    "style_emb": 200,
}

emb_cols = {
    "cos_img" : "img_emb",
    "cos_product": "product_emb",
    "cos_brand": "brand_emb",
    "cos_style": "style_emb",
    "cos_category": "category_emb",
    "cos_brand_desc": "brand_desc_emb",
    "cos_brand_cat": "brand_cat_emb",
}
result_df = df[["product_id", "product_name", "korea_name", "img_url"]].copy()

for cos_col, emb_col in emb_cols.items():

    # 기본값
    result_df[cos_col] = 0.0

    if emb_col not in df.columns:
        continue

    emb_dim = EMB_DIMS.get(emb_col)
    if emb_dim is None:
        continue

    # -------------------------------------------------
    # ① 컬럼별 정상 임베딩 마스크
    # -------------------------------------------------
    valid_mask = df[emb_col].apply(
        lambda x: isinstance(x, (list, np.ndarray)) and len(x) == emb_dim
    )

    if valid_mask.sum() == 0:
        continue

    # -------------------------------------------------
    # ② 정상 임베딩만 행렬화 + 정규화
    # -------------------------------------------------
    emb_matrix = np.vstack(df.loc[valid_mask, emb_col].values)
    emb_matrix = emb_matrix / (
        np.linalg.norm(emb_matrix, axis=1, keepdims=True) + 1e-9
    )

    # -------------------------------------------------
    # ③ 기준 상품 임베딩 추출
    # -------------------------------------------------
    if not valid_mask.iloc[target_idx]:
        continue

    valid_indices = df.index[valid_mask]
    target_pos = list(valid_indices).index(target_idx)
    target_vec = emb_matrix[target_pos]

    # -------------------------------------------------
    # ④ 코사인 유사도 → 원래 위치에 매핑
    # -------------------------------------------------
    sims = emb_matrix @ target_vec
    result_df.loc[valid_mask, cos_col] = sims

# =====================================================
# 4. 가중치 적용 및 최종 점수 계산
# =====================================================
weights = {
    "cos_product": 0.1, 
    "cos_category": 0.1, 
    "cos_style": 0.0,#안씀
    "cos_img": 1.0, 
    "cos_brand": 0.0,#안씀
    "cos_brand_desc": 0.05,
    "cos_brand_cat": 0.1
}

result_df["weighted_cosine"] = sum(
    result_df[col] * w for col, w in weights.items() if col in result_df.columns
)

# =====================================================
# 5. 결과 정리 (대표 상품 + TOP 30)
# =====================================================
rep_row = result_df[result_df["product_id"] == TARGET_ID].copy()
cand_df = result_df[result_df["product_id"] != TARGET_ID].copy()

top30_df = cand_df.sort_values("weighted_cosine", ascending=False).head(30)

# 시각화용 통합
final_display_df = pd.concat([rep_row, top30_df], ignore_index=True)

# =====================================================
# 6. HTML 갤러리 렌더링
# =====================================================
html_out = f"""
<div style="font-family: sans-serif; padding: 20px; background-color: #f4f7f6;">
    <h2 style="border-bottom: 3px solid #007bff; padding-bottom: 10px;">🛒 추천 시스템 결과</h2>
"""

def get_card(row, is_rep=False):
    img = row["img_url"] if pd.notnull(row["img_url"]) else "https://via.placeholder.com/150?text=No+Image"
    border = "3px solid #ff4b2b" if is_rep else "1px solid #ddd"
    badge = "<span style='background:#ff4b2b; color:white; padding:2px 6px; border-radius:4px; font-size:10px;'>대표</span>" if is_rep else ""

    return f"""
    <div style="width: 160px; border: {border}; border-radius: 12px; background: white; padding: 12px; text-align: center; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
        {badge}
        <div style="width: 100%; height: 180px; margin: 10px 0; overflow: hidden; border-radius: 8px; background: #eee;">
            <img src="{img}" style="width: 100%; height: 100%; object-fit: cover;" onerror="this.src='https://via.placeholder.com/150?text=Image+Error'">
        </div>
        <div style="font-size: 12px; font-weight: bold; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;">{row['product_name']}</div>
        <div style="font-size: 11px; color: #666; margin-top: 4px;">{row['korea_name']}</div>
        <div style="font-size: 11px; color: #007bff; font-weight: bold; margin-top: 6px;">Score: {row['weighted_cosine']:.4f}</div>
    </div>
    """

# 대표 상품 섹션
html_out += "<h3>⭐ 기준 상품</h3>"
html_out += "<div style='display: flex; gap: 15px; margin-bottom: 30px;'>"
html_out += get_card(rep_row.iloc[0], True)
html_out += "</div>"

# 추천 상품 섹션
html_out += "<h3>🚀 추천 결과 TOP 30</h3>"
html_out += "<div style='display: flex; flex-wrap: wrap; gap: 15px;'>"
for _, row in top30_df.iterrows():
    html_out += get_card(row)
html_out += "</div></div>"

display(HTML(html_out))

print(f"전체 실행 완료: {perf_counter() - start_total:.3f} sec")

In [ ]:
import pymysql
import numpy as np

# 1. DB 연결 설정
db_config = {
    "host": "YOUR_HOST",
    "user": "YOUR_USER",
    "password": "YOUR_PASSWORD",
    "database": "musinsa_db",
    "charset": "utf8mb4"
}

def update_null_embeddings(npz_path):
    # 2. npz 파일 로드
    data = np.load(npz_path)
    # npz 내부 키 이름은 실제 파일에 맞춰 수정하세요 (예: 'id', 'emb')
    npz_ids = data['ids'] 
    npz_embs = data['embeddings']
    
    # 조회를 빠르게 하기 위해 딕셔너리로 변환
    emb_dict = {int(idx): emb.astype(np.float32).tobytes() for idx, emb in zip(npz_ids, npz_embs)}

    conn = pymysql.connect(**db_config)
    try:
        with conn.cursor() as cursor:
            # 3. img_emb가 NULL인 id들만 조회
            print("결측치 ID 조회 중...")
            cursor.execute("SELECT id FROM product WHERE image_emb IS NULL")
            null_ids = [row[0] for row in cursor.fetchall()]
            
            if not null_ids:
                print("업데이트할 결측치가 없습니다.")
                return

            # 4. 업데이트 데이터 준비 (npz에 해당 ID가 있는 경우만)
            update_data = []
            for nid in null_ids:
                if nid in emb_dict:
                    # (바이너리데이터, id) 순서 -> UPDATE 쿼리의 %s 순서와 일치해야 함
                    update_data.append((emb_dict[nid], nid))

            # 5. executemany로 벌크 업데이트 실행
            if update_data:
                print(f"총 {len(update_data)}건 업데이트 시작...")
                
                # 배치 사이즈를 나누어 실행 (메모리 및 락 방지)
                batch_size = 5000
                sql = "UPDATE product SET image_emb = %s WHERE id = %s"
                
                for i in range(0, len(update_data), batch_size):
                    batch = update_data[i:i+batch_size]
                    cursor.executemany(sql, batch)
                    conn.commit()
                    print(f"{i + len(batch)}개 완료...")
                
                print("모든 결측치 업데이트 완료!")
            else:
                print("npz 파일 내에 일치하는 ID가 없습니다.")

    except Exception as e:
        print(f"에러 발생: {e}")
        conn.rollback()
    finally:
        conn.close()

# 실행
update_null_embeddings('path_to_your_file.npz')